# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# Import libraries
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from datetime import datetime

In [2]:
# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify Project") \
    .config("spark.driver.bindAddress","localhost")\
    .config("spark.ui.port","4050")\
    .getOrCreate()

In [3]:
# Check config
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '192.168.0.28'),
 ('spark.app.startTime', '1623916895449'),
 ('spark.sql.warehouse.dir',
  'file:/Users/michelangelo/Documents/dev/udacity/DSND_Term2_MR/CapstoneProject/notebooks/spark-warehouse'),
 ('spark.executor.id', 'driver'),
 ('spark.ui.port', '4050'),
 ('spark.app.id', 'local-1623916896556'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.bindAddress', 'localhost'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '50754'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'Sparkify Project')]

In [4]:
# Check session
spark

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
# Load data
# Mini dataset
path = "../data/mini_sparkify_event_data.json"
# Full dataset
# path ="s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
user_log = spark.read.json(path)

In [6]:
# check schema
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
# Describe
user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [8]:
# Show 1 row
# user_log.show(n=1)
user_log.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [ ]:
# Show 5 rows
# user_log.head(5)

In [9]:
# Check num of rows, columns
user_log.count(), len(user_log.columns)

(286500, 18)

In [10]:
# Check available levels
user_log.select("level").dropDuplicates().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [11]:
# Check available pages
user_log.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



In [13]:
# Filter users that went to the "Cancellation Confirmation" page
user_cancelling = user_log.select(["userId", "firstname"]).where(user_log.page == "Cancellation Confirmation").dropDuplicates().collect()

In [17]:
# How many are there?
len(user_cancelling)

52

In [20]:
# Take a look at the first
user_cancelling[0]

Row(userId='200021', firstname='Liliana')

In [34]:
# Check what happens to the first userID
user_0_lifecycle = user_log.select(["userId", "firstname", "page", "level", "ts"]).where(user_log.userId == user_cancelling[0][0]).collect()

In [35]:
len(user_0_lifecycle)

1502

In [36]:
user_0_lifecycle[-10:]

[Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541136496000),
 Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541136756000),
 Row(userId='200021', firstname='Liliana', page='Thumbs Down', level='paid', ts=1541136757000),
 Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541137368000),
 Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541137652000),
 Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541137957000),
 Row(userId='200021', firstname='Liliana', page='NextSong', level='paid', ts=1541138231000),
 Row(userId='200021', firstname='Liliana', page='Downgrade', level='paid', ts=1541138263000),
 Row(userId='200021', firstname='Liliana', page='Cancel', level='paid', ts=1541138264000),
 Row(userId='200021', firstname='Liliana', page='Cancellation Confirmation', level='paid', ts=1541138333000)]

In [41]:
# How much time did he/she spend with the service:
datetime.fromtimestamp(user_0_lifecycle[0][4]/1000), datetime.fromtimestamp(user_0_lifecycle[-1][4]/1000)

(datetime.datetime(2018, 10, 3, 18, 59, 19),
 datetime.datetime(2018, 11, 2, 6, 58, 53))

In [28]:
# take a look at the 10th
user_cancelling[9]

Row(userId='300007', firstname='Lucas')

In [42]:
# Check what happens to the 10th userID
user_10_lifecycle = user_log.select(["userId", "firstname", "page", "level", "ts"]).where(user_log.userId == user_cancelling[9][0]).collect()

In [43]:
len(user_10_lifecycle)

143

In [44]:
user_10_lifecycle[-10:]

[Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538704116000),
 Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538704422000),
 Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538704602000),
 Row(userId='300007', firstname='Lucas', page='Home', level='paid', ts=1538704615000),
 Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538704856000),
 Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538705054000),
 Row(userId='300007', firstname='Lucas', page='NextSong', level='paid', ts=1538705267000),
 Row(userId='300007', firstname='Lucas', page='Downgrade', level='paid', ts=1538705336000),
 Row(userId='300007', firstname='Lucas', page='Cancel', level='paid', ts=1538705337000),
 Row(userId='300007', firstname='Lucas', page='Cancellation Confirmation', level='paid', ts=1538705338000)]

In [45]:
# How much time did he/she spend with the service:
datetime.fromtimestamp(user_10_lifecycle[0][4]/1000), datetime.fromtimestamp(user_10_lifecycle[-1][4]/1000)

(datetime.datetime(2018, 10, 4, 20, 24, 56),
 datetime.datetime(2018, 10, 5, 4, 8, 58))

In [52]:
# Drop NaN in userId and sessionId
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])

In [53]:
user_log_valid.count()

286500

so none, really....

In [54]:
# take a look at the users...
user_log.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [55]:
# take a look at the sessions...
user_log.select("sessionId").dropDuplicates().sort("sessionId").show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



so some users are empty

In [56]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [57]:
user_log_valid.count()

278154

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.